In [12]:
# Imports, as always...
import numpy as np
import matplotlib.pyplot as plt 

# For generating prime numbers.
from Crypto.Util import number

from sqif import CVP, solve_cvp

# VQAs for CVP

A very important part of Yan et al. (2022)'s proposal for a quantum-accelerated variant of Schnorr's classical factoring algorithm is that an approximate solution to the closest vector problem (CVP) could be improved upon by considering a superposition over the states forming the unit hypercube around that solution, then using a variational algorithm (they use QAOA) to sample higher quality solutions from that search space.

The relevance of this within the picture of factoring is that we have, by Schnorr's classical factoring method, a sieve-based factoring method requiring sufficiently many "smooth-relation pairs (sr-pairs)" which we may found by reducing the problem to a closest vector problem (CVP) on the prime lattice whose elemental points map to sr-pairs. The higher the quality of our solution to the CVP, the 'better' our sr-pairs are more likely to be, and so we require fewer of them to form a suitable system of equations whose solution yields (part of) a solution to the original factorisation. 

Of course, Schnorr's algorithm is deeply flawed, but that is besides the point -- we know in what ways it is flawed, and why, thanks to works such as Léo Ducas' excellent repository. Our concern, in this notebook, is that *throwing the thing into a variational algorithm is not the silver bullet*; we suggest that Yan et al. (2022)'s proposal is built on a weak claim about lattice dimension, and so we do not expect better results even when using a quantum algorithm to improve the solution to the CVP. Indeed, we doubt their methodology is sophisticated enough to even improve the CVP effectively as the lattice dimension increases, particularly as the lattice dimension is given to scale "sublinearly" based on claims that are not expected to hold.

So, in summary, the purpose of this notebook is to empirically demonstrate that their variational approach, with a Hamiltonian mapped as they describe, offers an insufficient scaling with respect to the probability to sample better solutions to the CVP and lattice dimension, and thus their approach would not scale well even when issues with Schnorr's classical approach are not fundamental (which, unfortunately, they are).

**Hopefully, we can also try out a few other quantum heuristics (e.g. VQE, AQC-PQC), and maybe even a brute-force search of the entire hyper-cube to demonstrate that the idea of 'searching around the approximate solution on the off chance a rounding operation was not so good' is wholy insufficient to solve a problem as difficult as prime factorisation.**

In [17]:
# The lattice dimension we'd like to generate for.
lattice_dim = 5

# Generating a prime number for which the above lattice dimension is required.
N = number.getPrime(lattice_dim ** 2)

# Set up the CVP.
cvp = CVP()
cvp.generate_cvp(N, l=1, c=4, seed=42)

# Solve it!
solutions, probabilities, approximate_solution = solve_cvp(cvp, n_samples=1000, delta=.75, p=1, min_method='Nelder-Mead', verbose=False)